# 常微分方程的数值解法
就是计算精确解$y(x)$的近似解$y_n$，使得$y_n$在$x_n$处的误差$|y(x_n) - y_n|$尽可能小。
通常选等长的步长$h$，将区间$[a, b]$等分为$n$个小区间，$x_i = a + ih$，$i=0,1,\ldots,n$，$x_0=a$，$x_n=b$。

## 数值微分

数值微分是用离散的方法近似地计算函数导数的方法。通常仅当函数以离散数值形式给出时才有必要这么做。

- 前差分：$f'(x) \approx \frac{f(x+h) - f(x)}{h}$
- 后差分：$f'(x) \approx \frac{f(x) - f(x-h)}{h}$
- 中心差分：$f'(x) \approx \frac{f(x+h) - f(x-h)}{2h}$ （这是最常用的）

当函数在间距为h的n等分点$x_0\leq x_1\leq \ldots \leq x_n$上用离散数值表示为$y_0, y_1, \ldots, y_n$时，可以用中点公式计算导数：
$$
f'(x_i) \approx \frac{y_{i+1} - y_{i-1}}{2h}, \quad i=1,2,\ldots,n-1
$$
对于端点$x_0$和$x_n$，相应的计算公式为
$$
f'(x_0) \approx \frac{-3y_0 + 4y_1-y_2}{2h}, \quad f'(x_n) \approx \frac{3y_n - 4y_{n-1}+y_{n-2}}{2h}
$$
上述3式称为三点差分公式。误差为$O(h^2)$。

## 欧拉法

> Lipschitz条件：若函数$f(x, y)$满足$\left|f(x, y_1) - f(x, y_2)\right| \leq L\left|y_1 - y_2\right|$，则称$f(x, y)$满足Lipschitz条件，常数L称为Lipschitz常数。在满足Lipschitz条件下，解的存在唯一性定理成立，称为一阶常微分方程的初值问题。

欧拉法是一种常微分方程的数值解法。对于一阶微分方程$y'(x) = f(x, y)$，已知初始条件$y(x_0) = y_0$，欧拉法的基本思想是，在小区间$\[x_i,x_{i+1}\]$用前差分公式替代微分方程左边的导数，而右端的函数值$f(x, y(x))$用已知的$x_i$或$x_{i+1}$代替，从而得到一个递推公式：

- 向前欧拉法：$y_{n+1} = y_n + hf(x_n, y_n)$ 
  意思是从$x_0, y_0$开始，每次计算下一个点的y值，直到计算到$x_n$。

- 向后欧拉法：$y_{n+1} = y_n + hf(x_{n+1}, y_{n+1})$

向前欧拉公式称为显式公式，向后欧拉公式称为隐式公式。用后者计算复杂，通常用前者。

### 改进欧拉法

将向前欧拉法和向后欧拉法公式取平均，得到梯形公式：

$$
y_{n+1} = y_n + \frac{h}{2}(f(x_n, y_n) + f(x_{n+1}, y_{n+1}))
$$

显然这也是一种隐式公式，迭代过程计算量大。改进方法是用向前欧拉法预测$y_{n+1}$的近似值$\bar{y}_{n+1}$，用它代替梯形公式中的$y_{n+1}$，得到：
$$
\left\{
\begin{aligned}
\bar{y}_{n+1} &= y_n + hf(x_n, y_n) \\
y_{n+1} &= y_n + \frac{h}{2}(f(x_n, y_n) + f(x_{n+1}, \bar{y}_{n+1}))
\end{aligned}
\right.
$$

## Runge-Kutta法

### 算法实现

```python

import numpy as np
from scipy.integrate import solve_ivp

# 定义微分方程组函数
def ode_func(t, y, equations):
    # equations 是包含所有微分方程的函数列表
    dydt = np.array([equation(t, *y) for equation in equations])
    return dydt

# 定义微分方程组
def equation1(t, x, y):
    return x - 2 * y

def equation2(t, x, y):
    return x + y

equations = np.array([equation1, equation2])

# 初值条件
y0 = np.array([1.0, 0.0])  # 对应 x0 = 1.0, y0 = 0.0

# 求解时间范围
t_span = (0, 10)

# 使用 solve_ivp 来求解微分方程组
sol = solve_ivp(lambda t, y: ode_func(t, y, equations), t_span, y0, method='RK45')

# 获取解
t = sol.t
x, y = sol.y

# 打印结果
print("t:", t)
print("x:", x)
print("y:", y)


```

## 误差和精度的阶

### 局部截断误差

在迭代中，每一步的误差都会积累，最终导致整体误差。我们只研究一步的误差：局部截断误差。  
局部截断误差：在每一步计算中，由于用近似值代替精确值而产生的误差。
1. 将精确解在$x_n$处展开为泰勒级数：$y(x_{n+1}) = y(x_n) + h y'(x_n) + \frac{h^2}{2}y''(x_n) + \ldots$
2. 局部截断假定$y_n=y(x_n)$，代入公式得到估计值$y_{n+1}$，然后计算误差：$|y(x_{n+1}) - y_{n+1}|$
3. 以向前欧拉法为例，$y_{n+1} = y_n + hf(x_n, y_n)$，代入泰勒展开式，得到局部截断误差为$O(h^2)$。
4. 一般来说，欧拉法的局部截断误差为$O(h^2)$，梯形公式的局部截断误差为$O(h^3)$。
5. 精度的阶是指局部截断误差的阶低1阶，即欧拉法的精度为1，梯形公式的精度为2。

### 单步法的收敛性

各种显示单步法均可写成$y_{n+1} = y_n + h\varphi(x_n, y_n, h)$的形式，其中$\varphi(x_n, y_n, h)$是一个关于$x_n, y_n, h$的函数。称$\varphi(x_n, y_n, h)$为单步法的增量函数。
1. 若$\lim_{h\to 0}\varphi(x_n, y_n, h) = f(x_n, y_n)$，则称单步法是收敛的。

### 整体误差

单步法的收敛性中是指$n\to\infty,i.e.,h\to 0$时，$e_n = y(x_n) - y_n\to 0$。整体误差是指$e_n$的上界，即$|e_n| \leq Mh^p$，其中M是常数，p是精度的阶。\

### 稳定性

稳定性讨论的是迭代过程中，误差是否能被控制，形式上地讲
$$
\|\epsilon_{n+1}\| \leq \|\epsilon_n\|
$$

### 刚性问题

刚性问题是指微分方程中存在不同时间尺度的变量，导致数值解法的稳定性和精度受到影响。通常用刚性问题的解法是隐式方法。
- 快瞬态解：$\lambda$较大，变化快
  快瞬态解的特征根较大，如果用显式方法求解，步长$h$必须很小，计算量大。
- 慢瞬态解：$\lambda$较小，变化慢
  慢瞬态解的特征根较小，如果要精度较高，则需要计算的timesteps较多，计算量大。